# Image Preprocessing and Binary Classification with Keras

## Objective
In this week's exercise, you will:
1. Learn how to image preprocessing in keras.
2. Build and train a multilayer neural network for binary classification on a real-world dataset of cats and dogs.

---

## Step 1: Import Libraries
Let's start by importing the necessary libraries.


In [1]:
import numpy as np
import pathlib
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


---

## Step 2: Load and Preprocess the Data
We will use the Keras `ImageDataGenerator` for image augmentation and preprocessing.
First, unzip the uploaded dataset.


In [2]:
!unzip -q kagglecatsanddogs_5340.zip
# dataset_name = "cats_vs_dogs"
# (ds_train, ds_test), ds_info = tfds.load(
#     dataset_name,
#     split=['train[:80%]', 'train[80%:]'],  # Split into training and testing sets
#     shuffle_files=True,
#     as_supervised=True,  # Returns (image, label) pairs
#     with_info=True       # Includes dataset metadata
# )

# # Explore dataset info
# print(ds_info)

replace PetImages/Cat/0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


## Step 3: Learn about undersampling and implement it
Research online what undersampling and random undersampling is. It is a very powerful technique used often in machine Learning. Find out when it is used and undersample your dataset using "random undersampling"

In [3]:
data_dir = pathlib.Path("/content/PetImages")

# Load and preprocess the dataset
image_size = (180, 180)  # Match model input size
batch_size = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=image_size,
    batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=image_size,
    batch_size=batch_size)

# Normalize the dataset
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

# Collect all training images and labels for undersampling
all_images = []
all_labels = []

try:
    for images, labels in train_ds:
        all_images.append(images.numpy())
        all_labels.append(labels.numpy())
except Exception as e:
    print("error", e)

all_images = np.concatenate(all_images, axis=0)
all_labels = np.concatenate(all_labels, axis=0)

# Display the class distribution before undersampling
print(f"Class distribution before undersampling: {Counter(all_labels)}")

# Flatten images for undersampling
n_samples, height, width, channels = all_images.shape
reshaped_images = all_images.reshape(n_samples, -1)

# Perform undersampling
undersampler = RandomUnderSampler(random_state=42)
sampled_images, sampled_labels = undersampler.fit_resample(reshaped_images, all_labels)

# Reshape back to image dimensions
sampled_images = sampled_images.reshape(-1, height, width, channels)

# Display the class distribution after undersampling
print(f"Class distribution after undersampling: {Counter(sampled_labels)}")

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
error {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Input is empty.
	 [[{{node decode_image/DecodeImage}}]] [Op:IteratorGetNext] name: 
Class distribution before undersampling: Counter({1: 3664, 0: 3568})
Class distribution after undersampling: Counter({0: 3568, 1: 3568})


---

## Step 4: Set Up ImageDataGenerator (or well more specifically the new version)
Were Sorry - the videos from the coursera course are sometimes not the most up to date. In this case the 'ImageDataGenerator' function is deprecated (look here https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator) and will be removed in the future versions. The concept behind the new reccomended function is very similar though.
The new reccomendation is loading images with tf.keras.utils.image_dataset_from_directory and transforming the output tf.data.Dataset with preprocessing layers.

You may use Chat GPT for this task and you can also check the following tutorials <br>
https://www.tensorflow.org/tutorials/load_data/images <br>
https://www.tensorflow.org/tutorials/load_data/images <br>
https://www.tensorflow.org/guide/keras/preprocessing_layers <br>

In [ ]:
# TODO create a dataset using the recommended methods

---

## Step 5: Build a Multilayer Neural Network
Now, let's build a multilayer neural network for binary classification.


In [4]:
# Define the model
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=(180, 180, 3)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),  # Regularization
    tf.keras.layers.Dense(2, activation='softmax')
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


---

## Step 6: Train the Model
Train the model using the Dataset you created


In [ ]:
model.fit(sampled_images, sampled_labels, epochs=10, batch_size=batch_size, validation_data=val_ds)

Epoch 1/10
  3/223 ━━━━━━━━━━━━━━━━━━━━ 14:58 4s/step - accuracy: 0.5069 - loss: 39.0487

---

## Step 7: Evaluate the Model
After training, you may upload some test images to evaluate your model.


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
from google.colab import files

def load_and_predict(model):
    uploaded_files = files.upload()

    for fn in uploaded_files.keys():
        path = '/content/' + fn
        img = image.load_img(path, target_size=(150, 150))

        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0) / 255.0

        classes = model.predict(x)
        result = "a dog" if classes[0] > 0.5 else "a cat"

        print(f'The model predicts that the image is of {result}')

# Call the function to upload images and get predictions
load_and_predict(model)